In [1]:
import sys
!{sys.executable} -m pip install selenium undetected-chromedriver fake-useragent


  Using cached fake_useragent-2.1.0-py3-none-any.whl.metadata (17 kB)
Using cached fake_useragent-2.1.0-py3-none-any.whl (125 kB)



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
import time
import random

In [9]:


# 1️⃣ Hàm giả lập hành vi người dùng
def simulate_human_behavior(driver):
    action = ActionChains(driver)
    
    # Cuộn trang ngẫu nhiên
    for _ in range(random.randint(3, 6)):  # Cuộn 3-6 lần
        scroll_y = random.randint(400, 1200)  # Cuộn 400-1200 px
        driver.execute_script(f"window.scrollBy(0, {scroll_y});")
        time.sleep(random.uniform(2, 5))  # Nghỉ ngẫu nhiên 2-5s

    # Nhấp chuột ngẫu nhiên vào một số phần tử (nếu có)
    try:
        elements = driver.find_elements(By.XPATH, "//button | //div[not(ancestor::a) and not(@onclick)]")  
        if elements:
            element = random.choice(elements)
            action.move_to_element(element).click().perform()
            print("✅ Nhấp vào phần tử không điều hướng để giả lập hành vi người dùng.")
    except:
        print("⚠️ Không thể nhấp vào phần tử.")

# 2️⃣ Hàm truy cập trang chủ trước khi điều hướng
def access_homepage(driver, homepage_url):
    driver.get(homepage_url)
    time.sleep(random.uniform(5, 8))  # Chờ trang chủ tải
    simulate_human_behavior(driver)  # Giả lập hành vi trên trang chủ

# 3️⃣ Hàm crawl dữ liệu khách sạn
def crawl_data():
    homepage_url = "https://www.traveloka.com/vi-vn"
    hotel_url = "https://www.traveloka.com/vi-vn/hotel/search?spec=28-03-2025.29-03-2025.1.1.HOTEL_GEO.10010083.Đà%20Nẵng.1"

    # Cấu hình trình duyệt
    options = Options()
    options.add_argument("--disable-blink-features=AutomationControlled")  # Tránh bị phát hiện bot
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")
    
    # Khởi tạo trình duyệt với undetected-chromedriver
    driver = uc.Chrome(options=options)
    
    try:
        # Bước 1: Truy cập trang chủ trước
        access_homepage(driver, homepage_url)

        # Bước 2: Chuyển hướng đến trang khách sạn
        driver.get(hotel_url)
        time.sleep(random.uniform(5, 8))  # Chờ load trang khách sạn

        # Bước 3: Tiếp tục giả lập hành vi người dùng
        simulate_human_behavior(driver)

        # Bước 4: Lấy danh sách khách sạn
        hotel_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[class*='hotel-item']"))
        )
        print(f"✅ Tìm thấy {len(hotel_elements)} khách sạn!")

        hotels_data = []

        for hotel in hotel_elements[:5]:
            try:
                hotel_names = hotel.find_element(By.CSS_SELECTOR, "h3.hotel-name").text.strip()
            except:
                hotel_names = "N/A"

            try:
                location = hotel.find_element(By.CSS_SELECTOR, "div.hotel-address").text.strip()
            except:
                location = "N/A"

            try:
                price = hotel.find_element(By.CSS_SELECTOR, "div.price").text.strip()
            except:
                price = "N/A"

            try:
                score_hotels = hotel.find_element(By.CSS_SELECTOR, "div.hotel-rating").text.strip()
            except:
                score_hotels = "N/A"

            try:
                number_rating = hotel.find_element(By.CSS_SELECTOR, "div.review-count").text.strip()
            except:
                number_rating = "N/A"

            try:
                star_number = hotel.find_element(By.CSS_SELECTOR, "div.star-rating").text.strip()
            except:
                star_number = "N/A"

            try:
                received_time = hotel.find_element(By.CSS_SELECTOR, "div.check-in-time").text.strip()
            except:
                received_time = "N/A"

            try:
                giveback_time = hotel.find_element(By.CSS_SELECTOR, "div.check-out-time").text.strip()
            except:
                giveback_time = "N/A"

            try:
                description = hotel.find_element(By.CSS_SELECTOR, "div.hotel-description").text.strip()
            except:
                description = "N/A"

            try:
                hotel_link = hotel.find_element(By.CSS_SELECTOR, "a.hotel-link").get_attribute("href")
            except:
                hotel_link = "N/A"

            # Lưu dữ liệu vào danh sách
            hotels_data.append({
                "hotel_names": hotel_names,
                "location": location,
                "price": price,
                "score_hotels": score_hotels,
                "number_rating": number_rating,
                "star_number": star_number,
                "received_time": received_time,
                "giveback_time": giveback_time,
                "description": description,
                "hotel_link": hotel_link
            })

        # In kết quả
        for idx, hotel in enumerate(hotels_data, 1):
            print(f"\n🏨 Khách sạn {idx}:")
            print(f"   - Tên: {hotel['hotel_names']}")
            print(f"   - Địa điểm: {hotel['location']}")
            print(f"   - Giá: {hotel['price']}")
            print(f"   - Điểm đánh giá: {hotel['score_hotels']} ⭐")
            print(f"   - Số lượng đánh giá: {hotel['number_rating']} 📝")
            print(f"   - Xếp hạng sao: {hotel['star_number']} ✨")
            print(f"   - Check-in: {hotel['received_time']} ⏳")
            print(f"   - Check-out: {hotel['giveback_time']} ⏳")
            print(f"   - Mô tả: {hotel['description']}")
            print(f"   - Link: {hotel['hotel_link']} 🔗")

    except Exception as e:
        print("⚠️ Lỗi khi lấy dữ liệu:", e)

    # Đóng trình duyệt
    driver.quit()

# Gọi hàm crawl dữ liệu
crawl_data()


⚠️ Lỗi khi lấy dữ liệu: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=134.0.6998.177)
Stacktrace:
	GetHandleVerifier [0x00BCC7F3+24435]
	(No symbol) [0x00B52074]
	(No symbol) [0x00A206E3]
	(No symbol) [0x00A0FEC0]
	(No symbol) [0x00A2DFDF]
	(No symbol) [0x00A946AF]
	(No symbol) [0x00AAEB19]
	(No symbol) [0x00A8D5B6]
	(No symbol) [0x00A5C54F]
	(No symbol) [0x00A5D894]
	GetHandleVerifier [0x00ED70A3+3213347]
	GetHandleVerifier [0x00EEB0C9+3295305]
	GetHandleVerifier [0x00EE558C+3271948]
	GetHandleVerifier [0x00C67360+658144]
	(No symbol) [0x00B5B27D]
	(No symbol) [0x00B58208]
	(No symbol) [0x00B583A9]
	(No symbol) [0x00B4AAC0]
	BaseThreadInitThunk [0x75445D49+25]
	RtlInitializeExceptionChain [0x7758CDEB+107]
	RtlGetAppContainerNamedObjectPath [0x7758CD71+561]



In [ ]:
def process_rating(x):
    if x == 'No value':
        return x
    elif x == 'Chưa có đánh giá nào':
        return "No value"
    else:
        return x.split(' ')[3].replace('.', '')

def process_price(x):
    if x == 'No value':
        return x
    else:
        return x.split(' ')[0].replace('.', '')

def process_score(x):
    if x == '-':
        return "No value"
    else:
        return x
    
def get_url(list_of_places):
    """
    :param list_of_places: list of place you want to crawl. But need to take from url of traveloka
    :return: list of url from traveloka
    """
    list_of_url = []
    times = datetime.datetime.today().date()
    next_time = times + datetime.timedelta(days=1)
    for place in list_of_places:
        url = f"https://www.traveloka.com/vi-vn/hotel/search?spec={times.strftime(format='%d-%m-%Y')}.{next_time.strftime(format='%d-%m-%Y')}.1.1.HOTEL_GEO.{place}.1"
        list_of_url.append(url)
    return list_of_url
# 1. Hàm giả lập hành vi người dùng
def simulate_human_behavior(driver):
    action = ActionChains(driver)
    
    elements = driver.find_elements(By.XPATH, "//button | //div[not(ancestor::a) and not(@onclick)]")  
    if elements:
        element = random.choice(elements)
        try:
            action.move_to_element(element).click().perform()
            print("Nhấp vào phần tử không điều hướng")
        except:
            print("Không thể nhấp vào phần tử")

    # Cuộn trang
    driver.execute_script("window.scrollBy(0, 1000);")
    time.sleep(random.uniform(2, 5))
def crawl_data(list_of_url,list_of_places):
    options = Options()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")
    driver = uc.Chrome(options=options)

    print("🚀 ChromeDriver đã khởi chạy! Đang tải cookies...")
    driver.get("https://www.traveloka.com")
    #load_cookies(driver)
    time.sleep(5)
    driver.refresh()
    hotel_by_place = {}
    for url,place in zip(list_of_url,list_of_places):
        driver.get(url)
        simulate_human_behavior(driver)
        sleep(10)

        current_window = driver.current_window_handle

        def get_hotels():
            return driver.find_elements(By.XPATH, "//div[@class='css-1dbjc4n'][@data-testid='tvat-searchListItem']")

        df_traveloka = pd.DataFrame(columns=['hotel_names', 'location', 'price', 'score_hotels',
                                                'number_rating', 'star_number', 'received_time',
                                                'giveback_time', 'description', 'hotel_link'])
        list_hotel_exist = []
        hotel_names = []
        location_texts = []
        price_text = []
        description_text = []
        star_number_texts = []
        hotel_link = []
        score_hotels = []
        number_rating_text = []
        received_time = []
        giveback_time = []
        num_steps = 5000
        while True:

            previous_scroll_position = driver.execute_script("return window.scrollY")

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

            sleep(2)

            new_scroll_position = driver.execute_script("return window.scrollY")

            if new_scroll_position <= previous_scroll_position:
                break

        driver.execute_script("window.scrollTo(0, 0)")

        for step in range(num_steps):
            current_position = driver.execute_script("return window.scrollY")
            if current_position == new_scroll_position:
                break
            try:
                list_hotel = get_hotels()
                for index in range(len(list_hotel)):
                    if list_hotel[index] in list_hotel_exist:
                        continue

                    ActionChains(driver).move_to_element(list_hotel[index]).click().perform()

                    list_hotel_exist.append(list_hotel[index])

                    driver.switch_to.window(driver.window_handles[1])

                    wait_time = 10

                    try:
                        hotel_name_elements = WebDriverWait(driver, wait_time).until(
                            EC.visibility_of_all_elements_located((
                                By.XPATH,
                                "//h1[@class='css-4rbku5 css-901oao css-cens5h r-cwxd7f r-t1w4ow r-1ui5ee8 r-b88u0q r-nwxazl r-fdjqy7']"))
                        )
                    except:
                        print("Error hotel")
                        print("Link: ", driver.current_url)
                        print(hotel_name_elements)

                        try:
                            hotel_name_elements = WebDriverWait(driver, wait_time).until(
                                EC.visibility_of_all_elements_located((
                                    By.XPATH,
                                    "//h1[@class='css-4rbku5 css-901oao css-cens5h r-t1w4ow r-1ui5ee8 r-b88u0q r-nwxazl r-fdjqy7']"))
                            )
                        except:
                            print("Error hotel 2")
                            print("Link: ", driver.current_url)

                    try:
                        location_elements = WebDriverWait(driver, wait_time).until(
                            EC.visibility_of_all_elements_located((
                                By.XPATH,
                                "//div[@class='css-901oao css-cens5h r-cwxd7f r-13awgt0 r-t1w4ow r-1b43r93 r-majxgm r-rjixqe r-fdjqy7']"))
                        )
                    except:
                        print("Error location")
                        print("Link: ", driver.current_url)

                        try:
                            location_elements = WebDriverWait(driver, wait_time).until(
                                EC.visibility_of_all_elements_located((
                                    By.XPATH,
                                    "//div[@class='css-901oao css-cens5h r-13awgt0 r-t1w4ow r-1b43r93 r-majxgm r-rjixqe r-fdjqy7']"))
                            )
                        except:
                            print("Error location 2")
                            print("Link: ", driver.current_url)

                    try:
                        price_elements = WebDriverWait(driver, wait_time).until(
                            EC.visibility_of_all_elements_located((
                                By.XPATH, 
                                "//div[@class='css-901oao r-t1w4ow r-1x35g6 r-b88u0q r-vrz42v r-fdjqy7']"))
                        )
                    except:
                        print("Error in price")
                        print("Link: ", driver.current_url)

                    try:
                        star_number = WebDriverWait(driver, wait_time).until(
                            EC.visibility_of_all_elements_located((
                                By.XPATH,
                                "//div[@class='css-1dbjc4n']//div[@class='css-1dbjc4n r-18u37iz']//div[@class='css-1dbjc4n r-18u37iz']"))
                        )
                    except:
                        print("Error in star number")
                        print("Link: ", driver.current_url)

                    try:
                        score = WebDriverWait(driver, wait_time).until(
                            EC.visibility_of_all_elements_located((
                                By.XPATH,
                                "//div[@class='css-901oao r-jwli3a r-t1w4ow r-adyw6z r-b88u0q r-135wba7 r-fdjqy7']"))
                        )
                    except:
                        print("Error in score")
                        print("Link: ", driver.current_url)

                    try:
                        number_rating = WebDriverWait(driver, wait_time).until(
                            EC.visibility_of_all_elements_located((
                                By.XPATH,
                                "//div[@class='css-901oao r-jwli3a r-t1w4ow r-1enofrn r-b88u0q r-1cwl3u0 r-fdjqy7']"))
                        )
                    except:
                        print("Error in rating")
                        print("Link: ", driver.current_url)

                    try:
                        times = WebDriverWait(driver, wait_time).until(
                            EC.visibility_of_all_elements_located((
                                By.XPATH,
                                "//div[@class='css-901oao r-1h9nbw7 r-t1w4ow r-1b43r93 r-b88u0q r-rjixqe r-fdjqy7']"))
                        )
                    except:
                        print("Error in time")
                        print("Link: ", driver.current_url)

                    try:
                        for hotel_name_element in hotel_name_elements:
                            hotel_names.append(hotel_name_element.text)
                    except:
                        print("Error occurred while retrieving hotel names")

                    try:
                        for location_element in location_elements:
                            location_texts.append(location_element.text.split('\n')[0])
                    except:
                        print("Error location")

                    try:
                        for original_price_element in price_elements:
                            price_text.append(original_price_element.text)
                    except:
                        print("Error in price")

                    try:
                        for star_number_element in star_number:
                            star_number_texts.append(len(star_number_element.find_elements(By.TAG_NAME, "img")))
                    except:
                        print("Error star")

                    try:
                        for rating in number_rating:
                            number_rating_text.append(rating.text)
                    except:
                        print("Error in rating")

                    try:
                        for score_hotel in score:
                            score_hotels.append(score_hotel.text)
                    except:
                        print("Error in score")

                    try:
                        for received in times[::2]:
                            received_time.append(received.text)
                    except:
                        print("Error in received")

                    try:
                        for giveback in times[1::2]:
                            giveback_time.append(giveback.text)
                    except:
                        print("Error in giveback")

                    try:
                        hotel_link.append(driver.current_url)
                    except:
                        print("Error in link")

                    driver.execute_script(f"window.scrollTo(0, 20);")

                    try:
                        description_hotel = WebDriverWait(driver, wait_time).until(
                            EC.visibility_of_element_located((
                                By.XPATH,
                                "//div[@class='css-18t94o4 css-1dbjc4n r-kdyh1x r-1loqt21 r-10paoce r-5njf8e r-1otgn73 r-lrvibr']"))
                        )
                    except:
                        print("Error in button")

                    description_hotel.click()

                    detail_description = WebDriverWait(driver, wait_time).until(
                        EC.visibility_of_all_elements_located((
                            By.XPATH,
                            "//div[@class='css-1dbjc4n r-13awgt0 r-1rnoaur']//div[@class='css-1dbjc4n r-f4gmv6 r-nsbfu8']"))
                    )

                    for description in detail_description:
                        description_text.append(description.text)

                    driver.close()

                    driver.switch_to.window(current_window)
                    for check in [hotel_names, location_texts, price_text, 
                                    score_hotels, number_rating_text,star_number_texts,
                                    received_time, giveback_time, description_text, hotel_link]:
                        if check == []:
                            check.append("No value")

                    df_traveloka_new = pd.DataFrame(list(zip(hotel_names, location_texts, price_text,
                                                                score_hotels, number_rating_text, star_number_texts,
                                                                received_time, giveback_time, description_text, hotel_link)),
                                                    columns=['hotel_names', 'location', 'price', 'score_hotels',
                                                                'number_rating', 'star_number', 'received_time',
                                                                'giveback_time', 'description', 'hotel_link'])

                    df_traveloka = pd.concat((df_traveloka, df_traveloka_new), axis=0, ignore_index=True)

                    hotel_names = []
                    location_texts = []
                    price_text = []
                    description_text = []
                    star_number_texts = []
                    hotel_link = []
                    score_hotels = []
                    number_rating_text = []
                    received_time = []
                    giveback_time = []

            except Exception as e:
                print("Error:", e)
                print(driver.current_url)

        df_traveloka_nodup = df_traveloka.drop_duplicates('hotel_names').reset_index(drop=True)
        hotel_by_place[place] = df_traveloka_nodup
        hotel_by_place[place].to_csv(f"Hotel_{place}_Traveloka.csv",index=False)
    driver.quit()
    return hotel_by_place
def processing_data(list_of_dataframe):
    """
    :param list_of_dataframe: list of dataframe with key is place and value is dataframe
    :return: processed dataframe in csv file
    """
    processed_hotel = {}
    for place,data in zip(list_of_dataframe.keys(),list_of_dataframe.values()):
        data['number_rating'] = data['number_rating'].apply(process_rating)
        
        data['price'] = data['price'].apply(process_price)
        
        data['score_hotels'] = data['score_hotels'].apply(process_score)
        
        data['id'] = np.arange(1, len(data) + 1)
        
        data.to_csv(f"Processed_Data/Hotel_{place}_Processed.csv",index=False)
        
        processed_hotel[place] = data
    return processed_hotel

In [ ]:
if __name__ == "__main__":

    list_of_places = ['10010083.Đà%20Nẵng']

    list_of_url = get_url(list_of_places)
    hotel_by_place = crawl_data(list_of_places=list_of_places,list_of_url=list_of_url[:5])
    processing_hotel = processing_data(list_of_dataframe=hotel_by_place)

🚀 ChromeDriver đã khởi chạy! Đang tải cookies...
Nhấp vào phần tử không điều hướng
Error hotel
Link:  https://www.traveloka.com/vi-vn/hotel/detail?spec=27-03-2025.28-03-2025.1.1.HOTEL.1000000441652.Muong%20Thanh%20Grand%20Da%20Nang%20Hotel.1&contexts=%7B%22inventoryRateKey%22%3A%22tCI74RobDcii8TFR6EO8dfBoDgXZAdJDba%2F%2Bb%2Fyo7vnq%2BVVI3gI%2B2IwgtZZOU%2BVbqv6ET5ZPvddFMWIrecNvU5XL2WvGHaqiMknaaXu3aJUyHsGZcX44jhs%2BqFy07nDLzdPoEN8mT72XKB8FBTTfmaXK%2F%2FzWvKPRJXzfMzmIc82XoNF%2BfOpWhaXlFYj4va8QQ%2F9IIxUbseqUBFfJPq9iP5oSw3VdFi3k%2FxSn0NEC%2FCx9Ng75cf%2FD42KIBBOcs8eoOsjisiniA3%2B4ztxjOOTtaNYZkpJs9x4mTUm0vIF29wg1vZ5pYZKiymuJS5AJGaQB7NPBK53nSyOTT%2FOq8hX2GOPFYbIREQv81pQ2wBt10foU7KaNBRJ3pw8Z2O72QQ0ZhWDi4%2F413XozxeCOdOSJkHjo0U5esrcRm2v%2BNqY3MsnAFdBS0lpwxijvCuTQ23IeQPbR%2F2aOYj8R1PLW%2F5kTieLaz1cz9R7QS%2BlZLVFwC9Sc28KbyhcgN%2FTatq%2FtFzYvjZjtEkVdaIFEh%2FNSoLNOQeiKDIVSk%2BMO08K08W6f0NojJ9anWjNb4lqghsFNbIl9WSHoHbGSKNbI5Vp27E3I3n0uj490bSFOpUTghAekAIIktViWmzzVRDu8GkAF5LEw8kB1GNsKsgH0c2%2BOCjoxtCIcR4

OSError: Cannot save file into a non-existent directory: 'Processed_Data'